In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('CruisePrediction').getOrCreate()

In [3]:
df = spark.read.table('cruise_ship_info_csv')

In [4]:
from pyspark.ml.feature import StringIndexer

In [5]:
indexer = StringIndexer(inputCol='Cruise_line',outputCol='CruiseLineCategory')


In [6]:
indexed = indexer.fit(df).transform(df)

In [7]:
from pyspark.ml.linalg  import Vectors
from pyspark.ml.feature import VectorAssembler

In [8]:
indexed.columns


In [9]:
assembler = VectorAssembler(inputCols=['Age', 'Tonnage', 'passengers', 'length', 'cabins', 'passenger_density', 'CruiseLineCategory'],outputCol='features')

In [10]:
output = assembler.transform(indexed)

In [11]:
final_data = output.select('features','crew')

In [12]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [13]:
final_data.show()

In [14]:
from pyspark.ml.regression import LinearRegression

In [15]:
ship_lr = LinearRegression(labelCol='crew')

In [16]:
trained_ship_data = ship_lr.fit(train_data)

In [17]:
Results = trained_ship_data.evaluate(test_data)

In [18]:
Results.rootMeanSquaredError

In [19]:
Results.r2